# Generate Data

In [ ]:
import json
import networkx as nx
import itertools

## Load Pandemic board as graph

In [ ]:
pandemic_g = nx.read_graphml('pandemic.graphml.txt')

# dicts from city names to numbers, and from numbers to city names.
city_names_to_num = {tup[1]['label']:tup[0] for tup in pandemic_g.nodes(data=True)}
city_num_to_names = {tup[0]:tup[1]['label'] for tup in pandemic_g.nodes(data=True)}

## Define functions

In [3]:
def get_diameter(graph, stations):
    """Add edges between the stations and calculate the diameter."""
    
    # Make a copy of the graph because we're going to add edges.
    h = graph.copy()
    #print(stations)
    # Take the list of stations and add edges between them.
    for c in itertools.combinations(stations, 2):
        if c[0] in h and c[1] in h:
            h.add_edge(c[0], c[1])
        else:
            print('missing nodes' + c[0] + ' ' + c[1])
    # Replace nx.diameter() with an alternate metric that returns
    # a single value for the entire graph, i.e. average_shortest_path
    return (nx.diameter(h), stations, len(h.edges()))

In [ ]:
def get_neighbors(node):
    """Return set of node's neighbors and the node itself."""
    return set(nx.neighbors(pandemic_g, node)) | {node}

In [5]:
def get_node_combos(n, nodes, get_adj_func, starting_nodes=None):
    """Generator that produces combinations of non-adjacent research stations.
    
    Parameters
    ----------
    n : int
        Number of stations to add to the list of starting nodes.
    """
    # Create lists for storing stuff.
    cur_vals = [None for _ in xrange(n)]
    possible_vals = [[] for _ in xrange(n)]
    
    # If there are starting nodes, remove them and their neighbors from the set of possible values.
    if starting_nodes:
        for node in starting_nodes:
            nodes = set(nodes) - get_adj_func(node)

    # Set initial possible values.
    possible_vals[0] = nodes
    
    def next_level(possible_vals, cur_vals, level):
        if level == n-1:
            for a in possible_vals[level]:
                cur_vals[level] = a
                if starting_nodes:
                    yield starting_nodes + cur_vals[:]
                else:
                    yield cur_vals[:]
        else:
            for a in sorted(possible_vals[level]):
                cur_vals[level] = a
                possible_vals[level + 1] = {x for x in possible_vals[level] - get_adj_func(a) if x>a}
                for item in next_level(possible_vals, cur_vals, level + 1):
                    yield item

    for item in next_level(possible_vals, cur_vals, 0):
        yield item

In [6]:
list(get_node_combos(3, g.nodes(), get_neighbors, ['6']))

[['6', '1', '10', '30'],
 ['6', '1', '10', '28'],
 ['6', '1', '10', '22'],
 ['6', '1', '10', '29'],
 ['6', '1', '10', '24'],
 ['6', '1', '10', '25'],
 ['6', '1', '10', '26'],
 ['6', '1', '10', '20'],
 ['6', '1', '10', '21'],
 ['6', '1', '10', '48'],
 ['6', '1', '10', '23'],
 ['6', '1', '10', '46'],
 ['6', '1', '10', '47'],
 ['6', '1', '10', '44'],
 ['6', '1', '10', '45'],
 ['6', '1', '10', '42'],
 ['6', '1', '10', '43'],
 ['6', '1', '10', '40'],
 ['6', '1', '10', '41'],
 ['6', '1', '10', '3'],
 ['6', '1', '10', '4'],
 ['6', '1', '10', '7'],
 ['6', '1', '10', '13'],
 ['6', '1', '10', '15'],
 ['6', '1', '10', '14'],
 ['6', '1', '10', '17'],
 ['6', '1', '10', '16'],
 ['6', '1', '10', '19'],
 ['6', '1', '10', '32'],
 ['6', '1', '10', '31'],
 ['6', '1', '10', '49'],
 ['6', '1', '10', '36'],
 ['6', '1', '10', '35'],
 ['6', '1', '10', '33'],
 ['6', '1', '10', '18'],
 ['6', '1', '11', '30'],
 ['6', '1', '11', '28'],
 ['6', '1', '11', '29'],
 ['6', '1', '11', '24'],
 ['6', '1', '11', '25'],
 ['

## Generate data

In [ ]:
%%time
"""Generate data for 1 to 6 research stations."""
for i in xrange(1,6):
    results = [pandemic.get_diameter(pandemic_g, nodes) 
           for nodes in get_node_combos(i, g.nodes(), get_neighbors, starting_nodes=['6'])]
    json.dump(results, open(str(i) + '_nodes.json', 'wt'), encoding='utf8')